In [2]:
import csv
import time
import pandas as pd
from web3 import Web3
from tabulate import tabulate
from tqdm import tqdm

# Setup Web3 connection with your Infura API key, to get an API key goto: "https://www.infura.io/" for a free key
infura_url = "YOUR API KEY HERE"
web3 = Web3(Web3.HTTPProvider(infura_url))

# Oracle ABI with common price functions
oracle_function_abi = '''
[{
    "constant": true,
    "inputs": [],
    "name": "price",
    "outputs": [{"name": "", "type": "uint256"}],
    "type": "function"
},
{
    "constant": true,
    "inputs": [],
    "name": "latestAnswer",
    "outputs": [{"name": "", "type": "uint256"}],
    "type": "function"
}]
'''

# File path for 'morpho_vault_data.csv' produced by the "Pull Morpho Vault Data" notebook
csv_file_path = r"PATH TO \morpho_vault_data.csv"

# Read the CSV file to extract oracle addresses and vault names
oracle_data = []
with open(csv_file_path, mode='r') as file:
    reader = csv.DictReader(file)
    for row_num, row in enumerate(reader, start=1):
        oracle_data.append({
            'vault_name': row['vault_name'],
            'oracle_address': row['oracleAddress'],
            'row_num': row_num
        })

# Function to fetch oracle return and response latency
def fetch_oracle_return(oracle_address):
    try:
        if not web3.isAddress(oracle_address):
            return None
        
        oracle_contract = web3.eth.contract(address=oracle_address, abi=oracle_function_abi)
        
        try:
            # Try calling "price" function
            result = oracle_contract.functions.price().call()
            return result / 1e18  # Assume the price is in 1e18 and scale it
        except:
            try:
                # Try calling "latestAnswer" function
                result = oracle_contract.functions.latestAnswer().call()
                return result / 1e18  # Assume latestAnswer also needs scaling
            except Exception as e:
                return None
    except Exception as e:
        return None

# Query each oracle 5 times and take the average latency
def query_oracle_multiple_times(oracle_address, num_queries=5):
    total_latency = 0
    for _ in range(num_queries):
        start_time = time.time()
        fetch_oracle_return(oracle_address)
        latency = time.time() - start_time
        total_latency += latency
    return total_latency / num_queries

results = []

for entry in tqdm(oracle_data, desc="Querying Oracles"):
    vault_name = entry['vault_name']
    oracle_address = entry['oracle_address']
    row_num = entry['row_num']

    oracle_return = fetch_oracle_return(oracle_address)
    
    if oracle_return is not None:  # Only include rows with valid oracle returns
        average_latency = query_oracle_multiple_times(oracle_address)
        results.append({
            'Vault Name': vault_name,
            'Oracle Return': oracle_return, 
            'Average Response Latency (s)': average_latency,
            'Row #': row_num  # Row number from the the Morpho vault data CSV is stored here
        })

        
#The reason for the row number reference to the other CSV is because an oracle could be giving results for different 
#markets/token pairs. So this indexing is useful.
        
# Sort results by average latency in descending order
results = sorted(results, key=lambda x: x['Average Response Latency (s)'], reverse=True)

df = pd.DataFrame(results)

csv_output = pd.DataFrame(oracle_data)
csv_output['Oracle Return'] = df['Oracle Return']  
csv_output['Average Response Latency (s)'] = df['Average Response Latency (s)']
csv_output['Row #'] = df['Row #'] 
csv_output.to_csv('latency_report.csv', index=False)

# Output the results in a table format using tabulate, oracle address is excluded in the table but included in the .csv output
df = df.drop(columns=['Oracle Address'], errors='ignore')
print(tabulate(df, headers='keys', tablefmt='pretty'))


Querying Oracles: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 454/454 [02:35<00:00,  2.92it/s]

+-----+--------------------------------+------------------------+------------------------------+-------+
|     |           Vault Name           |     Oracle Return      | Average Response Latency (s) | Row # |
+-----+--------------------------------+------------------------+------------------------------+-------+
|  0  |        Steakhouse WBTC         |    4582390.44753596    |      0.0959784984588623      |  389  |
|  1  |       Gauntlet USDC Core       | 6.823153204063999e+20  |     0.08825478553771973      |  266  |
|  2  |       Gauntlet USDC Core       |     2776723889.01      |     0.08722100257873536      |  272  |
|  3  |        Steakhouse USDL         | 3.1234196894154794e+21 |     0.08711180686950684      |   2   |
|  4  |        Spark DAI Vault         |         1e+18          |     0.08666396141052246      |  196  |
|  5  |     LlamaRisk crvUSD Vault     | 9.017574247085214e+21  |     0.08593130111694336      |  337  |
|  6  |       Gauntlet WBTC Core       | 1.001201441730